In [2]:
TAVILY_API_KEY = 'tvly-KK'

In [3]:
import openfoodfacts
from pprint import pprint

# User-Agent is mandatory
api = openfoodfacts.API(user_agent="MyAwesomeApp/1.0")

In [4]:
result = api.product.text_search("Shrimp Flavour - Yum Yum", page=1, page_size=1)

In [5]:
pprint(result)

{'count': 10,
 'page': 1,
 'page_count': 1,
 'page_size': 1,
 'products': [{'_id': '8852018101055',
               '_keywords': ['asian-style',
                             'dehydrated',
                             'flavour',
                             'halal',
                             'noodle',
                             'oil',
                             'on',
                             'palm',
                             'roundtable',
                             'shrimp',
                             'soup',
                             'sustainable',
                             'thailand',
                             'with',
                             'yum',
                             'ผลิตภัณฑ์จากพืชทั้งหมด',
                             'พาสตา',
                             'อาหารแห้ง'],
               'added_countries_tags': [],
               'additives_debug_tags': [],
               'additives_n': 10,
               'additives_old_n': 9,
               'ad

In [7]:
pprint(result['products'][0]['ingredients_text_en'])

('WHEAT flour (GLUTEN), palm oil (palm oil, antioxidant: E306), salt, sugar, '
 'tapioca starch, modified tapioca starch, garlic, chilli 0.9%, flavour '
 'enhancers: E621, E635, acid: E330, maltodextrin, SOY sauce powder (SOYBEANS, '
 'glucose syrup, maltodextrin, colour: E150c), corn starch, FISH sauce '
 '(ANCHOVY extract [FISH], salt, sugar), seasoning (flavouring, maltodextrin, '
 'emulsifier: E322 [SOYBEAN], stabilizer: E414), onion, spring onion 0.1%, '
 'galangal 0.1%, SHRIMP (CRUSTACEANS) 0.1%, lemongrass 0.1%, anti-caking '
 'agent: E551')


In [ ]:
result['products'][0]['nutrient_levels_tags']

In [ ]:
nutriments = result['products'][0]['nutriments']
nutriments

In [ ]:
ingredients = result['products'][0]['ingredients']
ingredient_names = [ingredient['text'] for ingredient in ingredients]
ingri = ", ".join(ingredient_names)

ingri

In [ ]:
from langchain_upstage import ChatUpstage

In [ ]:
llm = ChatUpstage(api_key="up_x")

In [ ]:
from langchain_core.tools import tool
import requests
import os

@tool
def product_ingredients(query: str) -> str:
    """Based upon product name, get the product's detailed ingredients list."""
    result = api.product.text_search(query, page=1, page_size=1)
    ingredients = result['products'][0]['ingredients_text']
    # ingredients_tags = result['products'][0]['ingredients_tags']
    return ingredients

@tool
def get_nova_score(query: str) -> str:
    """Query for product Nova Group based upon product name. The Nova classification is a framework for grouping edible 
    substances based on the extent and purpose of food processing applied to them. Nova classifies 
    food into four groups: 
    1. Unprocessed or minimally processed foods
    2. Processed culinary ingredients
    3. Processed foods
    4. Ultra-processed foods
    """
    result = api.product.text_search(query, page=1, page_size=1)
    nova_groups = result['products'][0]['nova_groups_tags']
    return nova_groups

@tool
def get_nutriments(query: str) -> str:
    """This is a query for the product nutriment details per serving when product name is used as query. This would be helpful
    for anyone who wants to search for a particular product and wants all nutriments information."""
    result = api.product.text_search(query, page=1, page_size=1)
    nutriments = result['products'][0]['nutriments']
    nutriments_tags = result['products'][0]['nutrient_levels_tags']
    return nutriments, nutriments_tags





In [ ]:
tools = [product_ingredients, get_nova_score, get_nutriments]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
llm_with_tools.invoke("What nova group does this Buldak Hot Chicken Flavor Ramen product belong to? Is it safe to consume?").tool_calls

In [ ]:
def call_tool_func(tool_call):
    tool_name = tool_call["name"].lower()
    if tool_name not in globals():
        print("Tool not found", tool_name)
        return None
    selected_tool = globals()[tool_name]
    return selected_tool.invoke(tool_call["args"])

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt_template = PromptTemplate.from_template(
    """
    Please provide answer for question from the following context. 
    ---
    Question: {question}
    ---
    Context: {context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [ ]:
import os

def tool_rag(question):
    for _ in range(3): # try 3 times
        tool_calls = llm_with_tools.invoke(question).tool_calls
        if tool_calls:
            break
        else:
            print("try again")

    if not tool_calls:
        return "I'm sorry, I don't have an answer for that."
    
    print(tool_calls)
    context = ""
    for tool_call in tool_calls:
        context += str(call_tool_func(tool_call))

    chain = prompt_template | llm | StrOutputParser()
    return chain.invoke({"context": context, "question": question})

In [ ]:
pprint(tool_rag("What ingredients does this Buldak Hot Chicken Flavor Ramen product have? Is it safe to consume?"))